In [1]:
from docsumo_image_util.api import parse_pdf
import pandas as pd
import time
import copy
import os
import time
from typing import Union
import concurrent.futures
import threading




from pydantic import BaseModel, Field, field_validator
from abc import ABC, abstractmethod
from typing import Dict, List, Tuple
from docllm.caller.providers import Provider, ProviderFactory
from docllm.config import CallerConfig
from docllm.handlers import YamlFileHandler
from docllm.parser.input_parser import get_line_text
from docllm.parser.output_parser import parse_json
from sklearn.metrics import classification_report
from docllm.prompt.render import JinjaRenderer
from loguru import logger
from tenacity import retry, stop_after_attempt
from dotenv import load_dotenv
load_dotenv() 
# Create a lock for token updates
token_lock = threading.Lock()
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/media/veracrypt1/GAC.json"
# from app.config import config_by_name


2025-05-01 15:57:44.022 | WARNING  | docsumo_image_util.parse.ocr.google:<module>:47 - GOOGLE_APPLICATION_CREDENTIALS was not set. Make sure you have configured Application Default Credentials. 
2025-05-01 15:57:44.023 | WARNING  | docsumo_image_util.parse.ocr.google:<module>:50 - Set base64 of service account json as GOOGLE_APPLICATION_CREDENTIALS on docker. If local, set above env as json path
2025-05-01 15:57:44.025 | WARNING  | docsumo_image_util.parse.ocr.ms:<module>:29 - Azure Subscription key not found...
2025-05-01 15:57:44.026 | WARNING  | docsumo_image_util.parse.ocr.ms:<module>:33 - Azure Region not found...
2025-05-01 15:57:44.027 | WARNING  | docsumo_image_util.parse.ocr.ms:<module>:37 - Azure Computer Vision Endpoint not found...


In [2]:
class Classifer(ABC):
    """Base Classifier for Document Splitting."""

    @abstractmethod
    def _classify(self, *args, **kwargs):
        """
        Classifies a list of dataframe to it respective labels
        """
        raise NotImplementedError

    def classify(self, df: List[Dict]) -> Dict:
        """Main function where the df from process_image/request is passed into.

        Args:
            df (List): A list containing dictionary of OCR data

        Returns:
            Dict: A dictionary with pages, classifications and classified as a key.
        """
        if not df:
            print("[DEBUG] No data, returning empty.")
            return {"pages": [], "classifications": [], "classified": False}

        # # convert dataframe of whole document to page datraframe
        # df_pages = pd.DataFrame(df)
        # merged_output={}
        # strict_batched_df = self.split_strict_df(df_pages)
        # for single_batch in strict_batched_df:
        #     _pages = list(set(single_batch.page))
        #     df_lst = [single_batch[single_batch.page == idx] for idx in _pages]
        #     out = self._classify(df_lst)
        #     # Initialize keys if not present
        #     merged_output.setdefault("pages", []).extend(out.get("pages", []))
        #     merged_output.setdefault("classifications", []).extend(out.get("classifications", []))
        #     merged_output["is classified"] = merged_output.get("is classified", True) and out.get("is classified", True)
          
        # return merged_output

        # convert dataframe of whole document to page datraframe 
        df_pages = pd.DataFrame(df)
        _pages = list(set(df_pages.page))
        logger.info(f"The pages in the parsed document is {_pages}")
        df_lst = [df_pages[df_pages.page == idx] for idx in _pages]
        doc_type_lst, pg_chunk_lst, total_input_tokens, total_output_tokens = self._classify(df_lst)
        return doc_type_lst, pg_chunk_lst, total_input_tokens, total_output_tokens
    



In [3]:
def str_to_bool(v):
    return v.lower() in ("yes", "true", "t", "1")


class DoctypeConfig(BaseModel):
    doc_type_id: str = Field(..., description="Doctype ID")
    doc_type_title: str = Field(..., description="Doctype Title")
    prompt: str = Field("", description="Prompt for the doctype")


class RequestConfig(BaseModel):
    df: List[Dict] = Field(..., description="OCRed Dataframes for the file")
    doc_type_details: List[DoctypeConfig] = Field(..., description="Document Type Details")
    auto_classification_prompt: str = Field("", description="Prompt setup in auto_classify doctype")
    auto_classify: bool = Field(False, description="Auto classify flag")

    @field_validator("auto_classify")
    def convert_2_bool(cls, v):
        return str_to_bool(str(v))

In [37]:
DEFAULT_RES = {"pages": [], "classifications": [], "classified": True}

class LLMClassifier(Classifer):
    os.environ["LLM_PROVIDER"] = "openrouter"
    #pass your api_key here
    LLM_PROVIDER = ProviderFactory.create_provider(provider_name= os.environ.get("LLM_PROVIDER", "openai"), api_key=os.getenv("OPENROUTER_API_KEY"))

    def __init__(self, request_config: RequestConfig, model_name:str) -> None:
        #self.LLM_PROVIDER=ProviderFactory.create_provider(provider_name= os.environ.get("LLM_PROVIDER", "openai"), api_key=os.getenv("OPENROUTER_API_KEY"))
        self.request_config = request_config
        self.doc_types = [doc_type.doc_type_title for doc_type in request_config.doc_type_details]
        self.doc_types_mapper = {
            doc_type.doc_type_title: doc_type.doc_type_id for doc_type in request_config.doc_type_details
        }
        self.model_name = model_name
    def build_prompts(
        self, text_lst: List[str], product_prompt: str, is_auto_classify: bool, doc_types: Union[str, List[str]]
    ) -> Tuple[str, str]:
        config_name = "auto_classify_config" if is_auto_classify else "custom_doctype_config"
        config_path = f"objs/{config_name}.yaml"
        yaml_contents = YamlFileHandler(config_path).handle()
        system_prompt = yaml_contents.get("system_prompt", "")
        user_prompt = yaml_contents.get("user_prompt", "")
        final_user_prompt = JinjaRenderer().render(
            user_prompt,
            context={"ocr_text": text_lst, "doc_types": doc_types, "prompt": product_prompt},
        )
        return system_prompt, final_user_prompt


    def call_llm(self, llm_provider: Provider, user_prompt: str, system_prompt: str) -> Dict[str, str]:
        res = llm_provider.call(
            system_prompt=system_prompt,
            user_prompt=user_prompt,
            #write your model_name here
            caller_config=CallerConfig(model_name = self.model_name, allow_model= True),
        )

        if isinstance(res, str):
            output = res
        else:
            output = res.text
        return parse_json(output) , llm_provider.prompt_tokens, llm_provider.completion_tokens
    
    def create_chunk(self, classifier_out: List[Tuple]) -> Tuple[List, List]:
        """Creates chunk based on the classification output. It combines same doctype pages
        that are present adjancent to each other.

        Args:
            classifier_out (List[Tuple]): Each item in the list is the output from the
                `app.services.classify.classifier.DocClassifierV2._classify_df`.

        Returns:
            Tuple[List, List]: Tuple containing chunk lists for pages and doc_types.

        Example:
            Case 1:
            _______
            If the classifier_out:
            [
             ('form_1040___start', 1), ('form_1040', 2), ('form_1040___start', 3),
             ('form_1040', 4), ('form_8995a___start', 5), ('form_8995a', 6), ('acord25', 7), ('auto_classify', 8),
             ('acord25', 9), ('auto_classify', 10)
            ]

            `create_chunk` output is:
            (
              [
               [1, 2, 3, 4], [5, 6], [7], [8], [9], [10]
              ],
              [
               ['form_1040___start', 'form_1040', 'form_1040___start', 'form_1040'],
               ['form_8995a___start', 'form_8995a'],
               ['acord25'],
               ['auto_classify'],
               ['acord25'], ['auto_classify']
              ]
            )

            Case 2:
            _______
            [
             ('form_1040___start', 1), ('form_1040', 2),
             ('form_1040___start', 3),('form_1040', 4),
             ('form_8995a___start', 5), ('form_8995a', 6),
             ('form_1040___start', 7),('form_1040', 8),
            ]

            Output
            _______
            (
             [[1, 2, 3, 4], [5, 6], [7, 8]],
             [['form_1040___start', 'form_1040', 'form_1040___start', 'form_1040'],
              ['form_8995a___start', 'form_8995a'],
              ['form_1040___start', 'form_1040']
             ]
            )
        """
        n_pages = len(classifier_out)
        last_doc_type = classifier_out[0][0]

        page_chunk = []
        page_chunk_lst = []
        doc_types = []
        doc_types_lst = []
        start_keyword = "___start"
        chunked = 0  # flag to make sure if we miss out pages to chunk.

        for doc_type, page_num in classifier_out:
            if not last_doc_type == doc_type:
                if (last_doc_type + start_keyword == doc_type) or (doc_type + start_keyword == last_doc_type):
                    page_chunk.append(page_num)
                    doc_types.append(doc_type)
                    continue
                page_chunk = sorted(list(set(page_chunk)))
                chunked += len(page_chunk)
                page_chunk_lst.append(page_chunk)
                doc_types_lst.append(doc_types)
                page_chunk = [page_num]
                doc_types = [doc_type]
            else:
                page_chunk.append(page_num)
                doc_types.append(doc_type)
            last_doc_type = doc_type

        # check the flag and append the remaining pages chunks.
        if chunked != n_pages and page_chunk:
            page_chunk_lst.append(page_chunk)
            doc_types_lst.append(doc_types)

        # almost a dead code due to the above `chunked` flag
        # If all pages is auto_classify/delete or single doc_type
        if not page_chunk_lst:
            page_chunk_lst.append(list(range(1, n_pages + 1)))
            doc_types_lst.append([doc_type] * n_pages)

        last_chunk_page = page_chunk_lst[-1][-1]
        if last_chunk_page != n_pages:
            page_chunk = list(range(last_chunk_page + 1, n_pages + 1))
            doc_types_lst.append([classifier_out[-1][0]] * len(page_chunk))
            # page_chunk = sorted(list(set([last_chunk_page + 1, n_pages])))
            page_chunk_lst.append(page_chunk)
        return page_chunk_lst, doc_types_lst

    def _handle_auto_classify(self, text_lst: List[str]) -> Tuple[List, List]:
        """Handling split when upload doctype is auto classify"""
        
        system_prompt, user_prompt = self.build_prompts(
            text_lst, self.request_config.auto_classification_prompt, is_auto_classify=True, doc_types=self.doc_types
        )

        output, input_tokens, output_tokens = self.call_llm(LLMClassifier.LLM_PROVIDER, user_prompt, system_prompt)
        if not output:
            logger.info(f"[INFO] [Handle AutoClassify] LLM output is empty || llm_output: {output}")
            return [], []
        
        classification_output = [(el.get("label"), el.get("page")) for el in output.get("page_classifications")]
        logger.info(f"[INFO] Classification Output: {classification_output}")
        pg_chunk_lst, doc_types_lst = self.create_chunk(classification_output)
        logger.info(f"[INFO] After create_chunk, Page Chunks: {pg_chunk_lst}, Doc types: {doc_types_lst}")
        return pg_chunk_lst, doc_types_lst, input_tokens, output_tokens

    def _handle_custom_doctype(self, text_lst: List[str], product_prompt: str, doc_type: str) -> List[str]:
        """Handle split when upload doctype is other than auto classify"""
        system_prompt, user_prompt = self.build_prompts(
            text_lst, product_prompt, is_auto_classify=False, doc_types=doc_type
        )
        output = self.call_llm(LLMClassifier.LLM_PROVIDER, user_prompt, system_prompt)
        if not output:
            logger.info(f"[INFO] [Handle Custom Doctype] LLM output is empty || llm_output: {output}")
            return []
        doc_types_lst = [[el.get("label") for el in output.get("page_classifications")]]
        pg_chunk_lst = [[el.get("page") for el in output.get("page_classifications")]]
        # logger.info(f"[INFO] After custom doctype handling, Page Chunks: {pg_chunk_lst}, Doc types: {doc_types_lst}")
        return pg_chunk_lst, doc_types_lst
    
    def process_doc_type_classification(self, text_lst, single_batch_text, df_lst):
        if self.request_config.auto_classify:  # Auto-classification logic
            pg_chunk_lst, doc_types_lst, input_tokens_obtained, output_tokens_obtained = self._handle_auto_classify(single_batch_text)
            logger.info(f"In the process doctype classification the total input and output tokens are:{input_tokens_obtained} and {output_tokens_obtained}")
            if not pg_chunk_lst:
                return DEFAULT_RES           
            return pg_chunk_lst, doc_types_lst, input_tokens_obtained, output_tokens_obtained

        else:  # Single doctype logic
            doc_type_config = self.request_config.doc_type_details[0]
            if not doc_type_config.prompt:
                logger.info(f"[INFO] Prompt not found for {doc_type_config.doc_type_title} || Doctype config: {doc_type_config}")
                return DEFAULT_RES

            if len(text_lst) == 1:
                logger.info(f"[INFO] Skipping custom doctype split as it has single page || len(text_lst): {len(text_lst)}")
                return DEFAULT_RES

            default_pg_chunk_lst = [[df.page.unique().tolist()[0] + 1 for df in df_lst]]
            default_doc_types_lst = [[doc_type_config.doc_type_title] * len(default_pg_chunk_lst)]

            llm_pg_chunk_lst, llm_doc_types_lst = self._handle_custom_doctype(
                text_lst, doc_type_config.prompt, self.doc_types[0]
            )

            return llm_pg_chunk_lst or default_pg_chunk_lst, llm_doc_types_lst or default_doc_types_lst





In [38]:
class LLMNonBatched(LLMClassifier):
     
    def __init__(self, request_config: Union[Dict, RequestConfig], model_name:str) -> None:
        print("i am here")
        # request_config["doc_type_details"] = [
        #     DoctypeConfig(**item) for item in request_config["doc_type_details"]
        # ]
        request_config = RequestConfig(**request_config)
        super().__init__(request_config,model_name)  # This calls the parent class's __init__


    def _classify(self, df_lst: List[pd.DataFrame]) -> Dict[str, str]:

        text_lst = [get_line_text(df) if not df.empty else "EMPTY PAGE" for df in df_lst]
        start = time.time()
        pg_chunk_lst,doc_types_lst, total_input_tokens, total_output_tokens = self.process_doc_type_classification(text_lst,text_lst,df_lst)
        elapsed=time.time()-start
        logger.info(f"[INFO] The Total time taken for the non batched data is: {elapsed}")
        # pg_chunk_lst, doc_types_lst = LLMClassifier.process_chunk(pg_chunk_lst, doc_types_lst, merge_autoclassify=False)
        # logger.info(f"[INFO] After process_chunk, Page Chunks: {pg_chunk_lst}, Doc types: {doc_types_lst}")
        return doc_types_lst, pg_chunk_lst, total_input_tokens, total_output_tokens
    



In [39]:

class LLMStrictBatched(LLMClassifier):
    def __init__(self, request_config: dict, model_name: str) -> None:
        # Initialize RequestConfig from dict if needed
        cfg = RequestConfig(**request_config) if isinstance(request_config, dict) else request_config
        super().__init__(cfg, model_name)

    def split_strict_df(self, text_lst: List[str], batch_size: int = 25) -> List[List[str]]:
        """Split list of texts into strictly sized batches."""
        return [text_lst[i : i + batch_size] for i in range(0, len(text_lst), batch_size)]

    def _classify(self, df_lst: List[pd.DataFrame]) -> Tuple[List[str], List, int, int]:
        # 1. Extract text from DataFrames
        text_lst = [get_line_text(df) if not df.empty else "EMPTY PAGE" for df in df_lst]
        logger.info(f"The total length of obtained text list is {len(df_lst)} ")
        # 2. Batch the text
        batched_texts = self.split_strict_df(text_lst)
        num_batches = len(batched_texts)

        # Placeholder for ordered results and token counts
        results: List[tuple] = [None] * num_batches
        total_input_tokens = 0
        total_output_tokens = 0

        def run_batch(idx: int, batch: List[str]):
            """Wrapper to process one batch and tag with its index."""
            try:
                start = time.time()
                pg_chunks, doc_types, input_tokens, output_tokens = self.process_doc_type_classification(
                    text_lst, batch, df_lst
                )
                elapsed = time.time() - start
                logger.debug(f"Batch {idx} processed in {elapsed:.2f}s | Input tokens: {input_tokens}, Output tokens: {output_tokens}")
                return idx, pg_chunks, doc_types, input_tokens, output_tokens
            except Exception as e:
                logger.error(f"Error processing batch {idx}: {e}", exc_info=True)
                return idx, [], [], 0, 0  # Fallback to zero tokens

        # 3. Execute batches in parallel
        overall_start = time.time()
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(run_batch, i, batch) for i, batch in enumerate(batched_texts)]
            for future in concurrent.futures.as_completed(futures):
                idx, pg_chunks, doc_types, input_tokens, output_tokens = future.result()
                results[idx] = (pg_chunks, doc_types)
                with token_lock:
                    total_input_tokens = input_tokens
                    total_output_tokens = output_tokens

        logger.info(f"Processed {num_batches} batches in {time.time() - overall_start:.2f}s")
        logger.info(f"Total input tokens: {total_input_tokens}, Total output tokens: {total_output_tokens}")

        # 4. Flatten results in correct order
        all_pg_chunks: List = []
        all_doc_types: List = []
        for pg_chunks, doc_types in results:
            all_pg_chunks.extend(pg_chunks)
            all_doc_types.extend(doc_types)

        logger.info(f"After merging chunks: pages={len(all_pg_chunks)}, types={len(all_doc_types)}")

        #Returning tokens along with results
        return all_doc_types, all_pg_chunks, total_input_tokens, total_output_tokens



In [40]:
class LLMSpreadBatch(LLMClassifier):

    def __init__(self, request_config: dict, model_name:str):
        request_config = RequestConfig(**request_config)
        super().__init__(request_config,model_name)

    def split_spread_text(self, text_lst, batch_size=25, step=20):
        batched_texts = []

        for i in range(0, len(text_lst), step):
            end_batch_index = i + batch_size

            if(end_batch_index > len(text_lst) - 1):
                end_batch_index=len(text_lst)-1
                batch = text_lst[i : end_batch_index+1]
                batched_texts.append(batch)
                break

            batch = text_lst[i : end_batch_index]
            batched_texts.append(batch)
        print(f"The length of text_list  is:{len(text_lst)}")    
        print(f"The length of batched text is:{len(batched_texts)}")
        return batched_texts

    
    def _classify(self, df_lst: List[pd.DataFrame]) -> Tuple[List[str], List, int, int]:
        # Step 1: Extract text from all DataFrames
        text_lst = [get_line_text(df) if not df.empty else "EMPTY PAGE" for df in df_lst]

        start = time.time()
        
        # Step 2: Batch the text list
        batched_texts = self.split_spread_text(text_lst)
        num_batches = len(batched_texts)

        # Placeholder for ordered results and token counters
        results: List[tuple] = [None] * num_batches
        total_input_tokens = 0
        total_output_tokens = 0

        def run_batch(idx: int, batch: List[str]):
            """Wrapper to process one batch and tag with its index."""
            try:
                start = time.time()
                pg_chunks, doc_types, input_tokens, output_tokens = self.process_doc_type_classification(
                    text_lst, batch, df_lst
                )
                elapsed = time.time() - start
                logger.debug(f"Batch {idx} processed in {elapsed:.2f}s | Input Tokens: {input_tokens}, Output Tokens: {output_tokens}")
                return idx, pg_chunks, doc_types, input_tokens, output_tokens
            except Exception as e:
                logger.error(f"Error processing batch {idx}: {e}", exc_info=True)
                return idx, [], [], 0, 0

        # Step 3: Execute batches in parallel
        overall_start = time.time()
        with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
            futures = [executor.submit(run_batch, i, batch) for i, batch in enumerate(batched_texts)]
            for future in concurrent.futures.as_completed(futures):
                idx, pg_chunks, doc_types, input_tokens, output_tokens = future.result()
                results[idx] = (pg_chunks, doc_types)
                with token_lock:
                    total_input_tokens = input_tokens
                    total_output_tokens = output_tokens

        logger.info(f"Processed {num_batches} batches in {time.time() - overall_start:.2f}s")
        logger.info(f"Total input tokens: {total_input_tokens}, Total output tokens: {total_output_tokens}")

        # Step 4: Flatten results in correct order
        all_pg_chunks: List = []
        all_doc_types: List = []
        for pg_chunks, doc_types in results:
            all_pg_chunks.extend(pg_chunks)
            all_doc_types.extend(doc_types)

        logger.info(f"After merging chunks: pages={len(all_pg_chunks)}, types={len(all_doc_types)}")

        #Return all outputs and token counts
        return all_doc_types, all_pg_chunks, total_input_tokens, total_output_tokens

        

In [46]:
is_spread_batched:bool = False
is_strict_batched:bool = False
is_non_batched: bool = True
model_name="openai/gpt-4.1-mini"
BASE_DIR = os.getcwd()
test_folder_path = os.path.join(BASE_DIR, "test_single")
COST_MAPPING = {
        "meta-llama/llama-3-70b-instruct": [8.1e-7, 8.1e-7],
        "openai/gpt-4o": [0.000005, 0.000015],
        "openai/gpt-4-turbo": [0.00001, 0.00003],
        "anthropic/claude-3-opus": [0.000015, 0.000075],
        "google/gemini-flash-1.5": [2.5e-7, 7.5e-7],
        "qwen/qwen-2-72b-instruct": [5.9e-7, 7.9e-7],
        "anthropic/claude-3.5-sonnet": [0.000003, 0.000015],
        "openai/gpt-4.1-mini": [0.000003, 0.000012]
}


In [47]:
def run_classification_on_file(file_path):
    print(f"\n[INFO] Processing: {os.path.basename(file_path)}")
    
    df_digital, cdf_digital = parse_pdf(file_path)
    df_digital_dict = df_digital.to_dict(orient="records")

    # Defining your payload template 
    payload_template = {
        "auto_classify": True,
        "auto_classification_prompt": "Split each document type into separate pages.",
        "df": df_digital_dict,  
        "doc_type_details": [
            {
                "doc_type_id": "acord25",
                "doc_type_title": "Acord 25",
                "prompt": "Review every field extracted"
            },
            {
                "doc_type_id": "invoice",
                "doc_type_title": "Invoice",
                "prompt": "Review every field extracted"
            },
            {
                "doc_type_id": "form1040",
                "doc_type_title": "Form 1040",
                "prompt": "Review every field extracted."
            },
            {
                "doc_type_id": "form1040a",
                "doc_type_title": "Form 1040 A",
                "prompt": "Review every field extracted"
            },
            {
                "doc_type_id": "form1040b",
                "doc_type_title": "Form 1040 B",
                "prompt": "Review every field extracted"
            },
            {
                "doc_type_id": "w9",
                "doc_type_title": "W9",
                "prompt": "Review every field extracted."
            },
        ]
    }

    results = {}
  
    if is_non_batched:
        doc_types, chunks, total_input_tokens, total_output_tokens = LLMNonBatched(request_config=payload_template,model_name=model_name).classify(df=df_digital_dict)
        results["non_batched"] = [item for sub in doc_types for item in sub]
        results["total_input_tokens_non_batched"]= total_input_tokens
        results["total_output_tokens_non_batched"]= total_output_tokens
        input_rate, output_rate = COST_MAPPING[model_name]
        results["cost_non_batched"] = (total_input_tokens * input_rate) + (total_output_tokens * output_rate)

    if is_strict_batched:
        doc_types, chunks, total_input_tokens, total_output_tokens = LLMStrictBatched(request_config=payload_template, model_name=model_name).classify(df=df_digital_dict)
        results["strict_batched"] = [item for sub in doc_types for item in sub]
        results["total_input_tokens_strict_batched"]= total_input_tokens
        results["total_output_tokens_strict_batched"]= total_output_tokens
        input_rate, output_rate = COST_MAPPING[model_name]
        results["cost_strict_batched"] = (total_input_tokens * input_rate) + (total_output_tokens * output_rate)

    if is_spread_batched:
        doc_types, chunks, total_input_tokens, total_output_tokens = LLMSpreadBatch(request_config=payload_template,model_name=model_name).classify(df=df_digital_dict)
        results["spread_batched"] = [item for sub in doc_types for item in sub]
        results["total_input_tokens_spread_batched"] = total_input_tokens
        results["total_output_tokens_spread_batched"] = total_output_tokens
        input_rate, output_rate = COST_MAPPING[model_name]
        results["cost_spread_batched"] = (total_input_tokens * input_rate) + (total_output_tokens * output_rate)

    return results


In [48]:
# def remove_overlap_classifications(doc_type_lst_spread_batched, window_size=25, stride=5):
#     # Flatten the entire list of classifications
#     step = window_size - stride  # How much to move the window after each batch
#     batch_idx = 0
#     final_array = []
#     i = 0

#     while i < len(doc_type_lst_spread_batched):
#         # If we've reached the end of the current window
#         if i == batch_idx + window_size:
#             # Skip the overlap and move to the next batch
#             i += stride 
#             batch_idx += step + 5 
#             continue
        
#         # Otherwise, add the current item to final array
#         final_array.append(doc_type_lst_spread_batched[i])
#         i += 1

#     return final_array



def remove_overlap_classifications(doc_type_lst_spread_batched, window_size=25, stride=5):
    step = window_size - stride  # Effective jump after overlap
    final_array = []
    match_results = []
    mismatch_results = []
    i = 0
    batch_num = 0
    print(f"The total doc tyoe spread length is : {len(doc_type_lst_spread_batched)}")
    while i < len(doc_type_lst_spread_batched):
        # Define current window range
        window_start = i
        window_end = i + window_size
        # if(window_end > len(doc_type_lst_spread_batched)):
        #     window_end=len(doc_type_lst_spread_batched)
        current_window = doc_type_lst_spread_batched[window_start:window_end]

        # Add non-overlapping part to final array
        if i == 0:
            final_array.extend(current_window)
        else:
            final_array.extend(current_window[stride:])  # skip overlap

            # Compare overlap between previous and current window
            prev_overlap = doc_type_lst_spread_batched[window_start-5 : window_start]
            next_overlap = doc_type_lst_spread_batched[window_start : window_start+5]

            details = []
            for j, (prev, nxt) in enumerate(zip(prev_overlap, next_overlap)):
                if prev != nxt:
                    abs_index = window_start + j  # This is the page number in the full list
                    mismatch_results.append({
                        # "overlap_batch": batch_num,
                        "page_number": abs_index,
                        # "prev_value": prev,
                        # "next_value": nxt
                    })
            

        # Move to next window
        i += window_size
        batch_num += 1
    print(mismatch_results)

    return final_array, mismatch_results


In [49]:
def remove_overlap_pages(doc_type_lst_spread_batched, window_size=25, stride=5):
    # Flatten the entire list of pages

    step = window_size - stride  # How much to move the window after each batch
    batch_idx = 0
    final_array = []
    i = 0

    while i + window_size <= len(doc_type_lst_spread_batched):  # Ensure we don't go out of bounds
        # Add the indices of the pages in this batch (excluding the overlap)
        first_batch_indices = list(range(i, i + step))  # Get the indices of non-overlapping part
        final_array.extend(first_batch_indices)
        
        # Move the window forward by the full window size
        i += window_size

    # Add remaining indices if any
    final_array.extend(range(i, len(doc_type_lst_spread_batched)))

    return final_array



In [50]:
# Directory setup
output_dir = "./batching_and_splitting"
os.makedirs(output_dir, exist_ok=True)

cost_log_path = os.path.join(output_dir, "cost_log.txt")

def log_cost(batch_type, filename, input_tokens, output_tokens, cost):
    with open(cost_log_path, "a") as log_file:
        log_file.write(
            f"{model_name} | {filename} | {batch_type} | "
            f"Input Tokens: {input_tokens}, Output Tokens: {output_tokens}, "
            f"Cost: ${cost:.6f}\n"
        )


all_results = {}
ground_truth_files=[]
for filename in os.listdir(test_folder_path):
    if filename.endswith(".txt") and not filename.endswith("_log.txt"):
        full_path = os.path.join(test_folder_path, filename)
        txt_basename = os.path.splitext(filename)[0]
        ground_truth_files.append(txt_basename)
        
    if filename.endswith(".pdf"):
        full_path = os.path.join(test_folder_path, filename)
        pdf_basename = os.path.splitext(filename)[0]

        try:
            result = run_classification_on_file(full_path)
            all_results[filename] = result

            # Write non-batched results
            if is_non_batched:
                # print(f" Non-Batched - Input Tokens: {result['total_input_tokens_non_batched']}, "
                #     f"Output Tokens: {result['total_output_tokens_non_batched']}, "
                #     f"Cost: ${result['cost_non_batched']:.6f}")
                # log_cost("Non-Batched", filename, 
                #         result['total_input_tokens_non_batched'], 
                #         result['total_output_tokens_non_batched'], 
                #         result['cost_non_batched'])

                non_batched_path = os.path.join(output_dir, f"{pdf_basename}_non_batched.txt")
                with open(non_batched_path, "w") as file:
                    file.write("\n".join(result["non_batched"]))

            # Write strict-batched results
            if is_strict_batched:
                # print(f"  Strict-Batched - Input Tokens: {result['total_input_tokens_strict_batched']}, "
                #     f"Output Tokens: {result['total_output_tokens_strict_batched']}, "
                #     f"Cost: ${result['cost_strict_batched']:.6f}")
                # log_cost("Strict-Batched", filename, 
                #         result['total_input_tokens_strict_batched'], 
                #         result['total_output_tokens_strict_batched'], 
                #         result['cost_strict_batched'])

                strict_batched_path = os.path.join(output_dir, f"{pdf_basename}_strict_batched.txt")
                with open(strict_batched_path, "w") as file:
                    file.write("\n".join(result["strict_batched"]))

            # Write spread-batched results (with overlap removed)
            if is_spread_batched:
                # print(f"  Spread-Batched - Input Tokens: {result['total_input_tokens_spread_batched']}, "
                #     f"Output Tokens: {result['total_output_tokens_spread_batched']}, "
                #     f"Cost: ${result['cost_spread_batched']:.6f}")
                # log_cost("Spread-Batched", filename, 
                #         result['total_input_tokens_spread_batched'], 
                #         result['total_output_tokens_spread_batched'], 
                #         result['cost_spread_batched'])

                spread_cleaned, unmatching_result = remove_overlap_classifications(result["spread_batched"])

                all_results[filename]["spread_batched"] = spread_cleaned
                all_results[filename]["unmatching_result"] = unmatching_result
                spread_batched_path = os.path.join(output_dir, f"{pdf_basename}_spread_batched.txt")
                with open(spread_batched_path, "w") as file:
                    file.write("\n".join(spread_cleaned))

        except Exception as e:
            print(f"[ERROR] Failed on {filename}: {e}")


2025-05-01 17:05:04.050 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.055 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.091 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.096 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is None
2025-05-01 17:05:04.120 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.124 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is None



[INFO] Processing: merged_output_10.pdf


2025-05-01 17:05:04.139 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is None
2025-05-01 17:05:04.168 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is None
2025-05-01 17:05:04.218 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.246 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.272 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.288 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.293 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is 0
2025-05-01 17:05:04.297 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is None
2025-05-01 17:05:04.322 | INFO     | docsumo_image_util.parse.pdf.parse:_orient_df:64 - orientation is None
2025-05-01 17:05:04.345 | INFO     | docsum

i am here


2025-05-01 17:05:10.891 | INFO     | __main__:classify:42 - The pages in the parsed document is [0, 2, 4, 6, 7, 8, 9, 10, 11, 14, 15, 16, 17, 20, 22, 23, 24, 25, 26, 27, 28, 29, 32, 33, 34, 36, 38, 40, 41, 42, 43, 44, 45, 48, 49, 50, 51, 54, 56, 57, 58, 59, 60, 61, 62, 63, 66, 67, 68, 70, 72, 74, 75, 76, 77, 78, 79, 82, 83, 84, 85, 88, 90, 91, 92, 93, 94, 95, 96, 97, 100, 101, 102, 104, 106, 108, 109, 110, 111, 112, 113, 116, 117, 118, 119, 122, 124, 125, 126, 127, 128, 129, 130, 131, 134, 135, 136, 138, 140, 142, 143, 144, 145, 146, 147, 150, 151, 152, 153, 156, 158, 159, 160, 161, 162, 163, 164, 165, 168, 169, 170, 172, 174]
2025-05-01 17:05:12.534 | INFO     | docllm.caller.base:_call_partial:122 - [INFO] Using OpenRouterProvider with model openai/gpt-4.1-mini, allow_model: True and params {'temperature': 0.2, 'frequency_penalty': 0, 'presence_penalty': 0}.
2025-05-01 17:05:46.317 | INFO     | __main__:_handle_auto_classify:158 - [INFO] Classification Output: [('Acord 25', 1), ('For

In [ ]:
# with open("test.txt", "w") as file:
#     file.write("\n".join(all_results["abc_merged.pdf"]["spread_batched"]))


all_results["new_merged.pdf"]["matching_result"]

[]

In [67]:
from sklearn.metrics import classification_report
import os

# Directories
predicted_dir = "./batching_and_splitting"
ground_truth_dir = test_folder_path  # Already defined earlier

# Batch types to evaluate
batching_modes = ["non_batched", "strict_batched", "spread_batched"]

# Collect all ground truth files
ground_truth_files = [f for f in os.listdir(ground_truth_dir) if f.endswith(".txt")]

# Prepare accumulators for full evaluation
all_y_true = {mode: [] for mode in batching_modes}
all_y_pred = {mode: [] for mode in batching_modes}

# Evaluation loop per file
for gt_file in ground_truth_files:
    gt_basename = os.path.splitext(gt_file)[0]
    gt_path = os.path.join(ground_truth_dir, gt_file)

    with open(gt_path, "r") as f:
        y_true = [line.strip() for line in f if line.strip()]

    print(f"\nEvaluating: {gt_file}")

    for batch_type in batching_modes:
        pred_filename = f"{gt_basename}_{batch_type}.txt"
        pred_path = os.path.join(predicted_dir, pred_filename)

        if not os.path.exists(pred_path):
            print(f"Missing prediction file for {batch_type}: {pred_filename}")
            continue

        with open(pred_path, "r") as f:
            y_pred = [line.strip() for line in f if line.strip()]

        if len(y_pred) != len(y_true):
            print(f"Length mismatch for {batch_type} on {gt_file} | y_true: {len(y_true)}, y_pred: {len(y_pred)}")
            continue

        # Append to the all-level list for later summary
        all_y_true[batch_type].extend(y_true)
        all_y_pred[batch_type].extend(y_pred)

        # Individual report
        report = classification_report(y_true, y_pred, zero_division=0)
        print(f"\nClassification Report for: {batch_type}")
        print(report)

        # Save per-file report
        model_name_clean = model_name.replace("/", "_")
        report_save_path = os.path.join(predicted_dir, f"{gt_basename}_{batch_type}_{model_name_clean}_report.txt")
        with open(report_save_path, "w") as f:
            f.write(report)

# --- Overall reports ---
print("\n===== OVERALL BATCH-TYPE REPORTS =====\n")

for batch_type in batching_modes:
    if all_y_true[batch_type] and all_y_pred[batch_type]:
        overall_report = classification_report(all_y_true[batch_type], all_y_pred[batch_type], zero_division=0)
        print(f"\n=== Overall Report for {batch_type} ===\n")
        print(overall_report)

        # Save to file
        overall_report_path = os.path.join(predicted_dir, f"overall_{batch_type}_{model_name_clean}_report.txt")
        with open(overall_report_path, "w") as f:
            f.write(overall_report)
    else:
        print(f"\nNo data to generate report for {batch_type}.")



Evaluating: new_merged.txt
Missing prediction file for non_batched: new_merged_non_batched.txt
Missing prediction file for strict_batched: new_merged_strict_batched.txt

Classification Report for: spread_batched
              precision    recall  f1-score   support

    Acord 25       1.00      1.00      1.00        23
   Form 1040       1.00      1.00      1.00        50
     Invoice       1.00      1.00      1.00        27

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100


Evaluating: abc_merged.txt
Missing prediction file for non_batched: abc_merged_non_batched.txt
Missing prediction file for strict_batched: abc_merged_strict_batched.txt

Classification Report for: spread_batched
              precision    recall  f1-score   support

    Acord 25       1.00      1.00      1.00        20
   Form 1040       1.00      1.00      1.00        60
     Invoice       1.00      1